In [2]:
%%capture
!pip install tensorflow=='2.3.0' keras=='2.3.1' numpy=='1.18.5'
!pip install ydata-profiling --upgrade

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU
import keras

import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd

import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

%matplotlib inline
import matplotlib as mpl
mpl.rc("figure, fgsize=(16, 9)")

<ipython-input-3-fbda8c563aab>:13: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [4]:
# Método generado para permitir que todas las ejecuciones terminen en una misma ejecución
def do_seeds(sn):
    os.environ['PYTHONHASHSEED']=str(sn)
    np.random.seed(sn)
    tf.random.set_seed(sn)
    random.seed(sn)

do_seeds(0)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# DATASET LOCATION: https://www.kaggle.com/pablomonleon/montreal-bike-lanes
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/University/Deep Learning/EjemploSimilarEntrega/airfoil_self_noise.csv')
numero_ejemplos = len(df)

# Divido el Dataset en 80% para train y 20% para test
train_df = df.head(int(numero_ejemplos*.8)).copy()
test_df = df.tail(int(numero_ejemplos*.2)).copy()

In [8]:
print(df.shape)
df.head()

(1503, 7)


Frequency  Angle of attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level   nominal  
0                             0.002663                      126.201  addition  
1                             0.002663                      125.201  addition  
2                             0.002663                      125.951  addition  
3                             0.002663                      127.591  addition  
4                             0.002663                      127.461  addition

1. Análisis exploratorio: identificar valores desconocidos, filas duplicadas y estadísticos de las variables del conjunto de datos


Según el informe, no hay valores desconocidos (NAs), ni filas duplicadas. Los estadísticos aparecen junto con cada distribución.

In [9]:
profile = ProfileReport(train_df)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

3. Eliminar las variables de entrada no numéricas

In [10]:
print(train_df.shape, test_df.shape)
train_df = train_df.select_dtypes(['number'])
test_df = train_df.select_dtypes(['number'])
print(train_df.shape, test_df.shape)

(1202, 7) (300, 7)
(1202, 6) (1202, 6)


5. Normalizar variables de entrada mediante min-max

**Nota:** Para que funcionen correctamente las Redes Neuronales, se deben normalizan tanto las variables explicativas (X) como la variable a predecir (Y)

In [11]:
# Se separan las variables explicativas de las variables a predecir
from sklearn.model_selection import train_test_split

target_column = "Scaled sound pressure level"
X = train_df.drop(target_column, axis=1)
y = train_df[target_column].to_numpy().reshape((-1,1))

X_test = test_df.drop(target_column, axis=1)
y_test = test_df[target_column].to_numpy().reshape((-1,1))

In [12]:
from sklearn.preprocessing import MinMaxScaler

# Se transforman las variables explicativas (features)
scaler_x = MinMaxScaler()
scaler_x.fit(X)
X = scaler_x.transform(X)
X_test = scaler_x.transform(X_test)

In [13]:
# Se comprueba que todas las variables están en el rango de valores deseado
np.min(X, axis=0), np.max(X, axis=0)

(array([0., 0., 0., 0., 0.]), array([1., 1., 1., 1., 1.]))

In [14]:
# Se transforman la variable a predecir (Target)
scaler_y = MinMaxScaler()
scaler_y.fit(y)
y = scaler_y.transform(y)
y_test = scaler_y.transform(y_test)

In [15]:
# Se comprueba que todas las variables están en el rango de valores deseado
np.min(y, axis=0), np.max(y, axis=0)

(array([0.]), array([1.]))

6. Dividir los datos de entrenamiento en Train (80%) y Validation (20%)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print('X Original:',X.shape,'; X train:',X_train.shape,'; X test:',X_val.shape)

X Original: (1202, 5) ; X train: (961, 5) ; X test: (241, 5)


7. Crear una Red Neuronal con 1 capas ocultas, 150 neuronas en cada capa y función de activación ReLu

**Nota**: La siguiente Red Neuronal tiene:
- Input: 5 datos
- Hidden Layer 1: 150 neuronas
- Output Layer: 1 neurona (rango de valores posibles: -infinito a +infinito)

**Importante**, revisad que estáis introduciendo todas las layer deseadas (incluyendo input, hidden y output)

In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.utils import to_categorical

model = Sequential()
model.add(Dense(150, input_shape=(5,) , activation='relu'))
model.add(Dense(1, activation=None))

8. Entrenar el algoritmo utilizando la métrica RMSE como función de coste

**Nota:** Importante, resulta necesario identificar si el problema que se está resolviendo es de clasificación o de regresión
  - Un problema de **clasificación** es un problema donde se quiere etiquetar un ejemplo en una categoría. Imaginate que tienes un problema donde quieres predecir si un cuadro es de sobre un perro, un gato o ninguno de los dos.
  - Un problema de **regresión** es un problema donde se quiere predecir un valor numérico que va desde menos infinito a más infinito. Por ejemplo, se desea predecir el año en el que un cuadro se pintó. El cuadro podría ser de hace 1.000.000 de años o podría ser del año 1.800.

In [18]:
model.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])

model.fit(X_train,y_train, epochs=7, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/7
31/31 [==============================] - 1s 6ms/step - loss: 0.2304 - mean_absolute_error: 0.4200 - val_loss: 0.0719 - val_mean_absolute_error: 0.2233
Epoch 2/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0638 - mean_absolute_error: 0.2018 - val_loss: 0.0571 - val_mean_absolute_error: 0.1962
Epoch 3/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0489 - mean_absolute_error: 0.1816 - val_loss: 0.0433 - val_mean_absolute_error: 0.1710
Epoch 4/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0372 - mean_absolute_error: 0.1580 - val_loss: 0.0349 - val_mean_absolute_error: 0.1555
Epoch 5/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0277 - mean_absolute_error: 0.1370 - val_loss: 0.0256 - val_mean_absolute_error: 0.1307
Epoch 6/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0214 - mean_absolute_error: 0.1203 - val_loss: 0.0210 - val_mean_absolute_error: 0.1178
Epoch 7/7
31/31 [=================

9. Re-Entrenar el algoritmo evitando predicciones con precios inferiores a 0€ (utilizar función de activación)

Nota: La activación ReLu transforma cualquier predicción al rango (0,infinito]

In [19]:
model2 = Sequential()
model2.add(Dense(150, input_shape=(5,), activation='relu'))
model2.add(Dense(1, activation='relu'))

model2.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
model2.fit(X_train,y_train, epochs=7, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/7
31/31 [==============================] - 1s 6ms/step - loss: 0.3667 - mean_absolute_error: 0.5769 - val_loss: 0.3894 - val_mean_absolute_error: 0.5968
Epoch 2/7
31/31 [==============================] - 0s 2ms/step - loss: 0.3667 - mean_absolute_error: 0.5769 - val_loss: 0.3894 - val_mean_absolute_error: 0.5968
Epoch 3/7
31/31 [==============================] - 0s 2ms/step - loss: 0.3667 - mean_absolute_error: 0.5769 - val_loss: 0.3894 - val_mean_absolute_error: 0.5968
Epoch 4/7
31/31 [==============================] - 0s 2ms/step - loss: 0.3667 - mean_absolute_error: 0.5769 - val_loss: 0.3894 - val_mean_absolute_error: 0.5968
Epoch 5/7
31/31 [==============================] - 0s 3ms/step - loss: 0.3667 - mean_absolute_error: 0.5769 - val_loss: 0.3894 - val_mean_absolute_error: 0.5968
Epoch 6/7
31/31 [==============================] - 0s 2ms/step - loss: 0.3667 - mean_absolute_error: 0.5769 - val_loss: 0.3894 - val_mean_absolute_error: 0.5968
Epoch 7/7
31/31 [=================

10. Evaluar la predicción en Test

In [20]:
print('[RMSE, Mean Absolute Error]')
model.evaluate(X_test,y_test)

[RMSE, Mean Absolute Error]
38/38 [==============================] - 0s 843us/step - loss: 0.0169 - mean_absolute_error: 0.1046


[0.01693037897348404, 0.1046389564871788]

11. Crear arquitectura que produzca overfit

- En este caso, con este conjunto de datos, no se consegue overfit debido a que el conjunto de datos es muy simple.
  - Normalmente, lo que sucedería es que el modelo continuaría reduciendo el error en Train, mientras el error en test después de unas épocas, empezaría a aumentar.

In [21]:
model_overfit = Sequential()
model_overfit.add(Dense(900, input_shape=(5,), activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(900, activation='relu'))
model_overfit.add(Dense(1, activation='relu'))

model_overfit.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
model_overfit.fit(X_train,y_train, epochs=7, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/7
31/31 [==============================] - 3s 48ms/step - loss: 0.1125 - mean_absolute_error: 0.2621 - val_loss: 0.0284 - val_mean_absolute_error: 0.1350
Epoch 2/7
31/31 [==============================] - 1s 43ms/step - loss: 0.0199 - mean_absolute_error: 0.1129 - val_loss: 0.0150 - val_mean_absolute_error: 0.0958
Epoch 3/7
31/31 [==============================] - 2s 59ms/step - loss: 0.0124 - mean_absolute_error: 0.0860 - val_loss: 0.0114 - val_mean_absolute_error: 0.0806
Epoch 4/7
31/31 [==============================] - 2s 68ms/step - loss: 0.0107 - mean_absolute_error: 0.0794 - val_loss: 0.0122 - val_mean_absolute_error: 0.0908
Epoch 5/7
31/31 [==============================] - 1s 44ms/step - loss: 0.0111 - mean_absolute_error: 0.0836 - val_loss: 0.0083 - val_mean_absolute_error: 0.0724
Epoch 6/7
31/31 [==============================] - 1s 44ms/step - loss: 0.0076 - mean_absolute_error: 0.0643 - val_loss: 0.0103 - val_mean_absolute_error: 0.0790
Epoch 7/7
31/31 [===========

12. Probar 3 ejemplos con distintas regularizaciones y identificar la que mejor funciona

**Nota:** El mejor modelo es que el tiene el menor error en los datos de validación, no entrenamiento. En este caso, el Dropout es el mejor modelo entre los 3.

- Regularizador L1

In [22]:
from tensorflow.keras import regularizers

model_reg1 = Sequential()
model_reg1.add(Dense(150, input_shape=(5,), activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.02)))
model_reg1.add(Dense(1, activation='relu'))

model_reg1.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
model_reg1.fit(X_train,y_train, epochs=7, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/7
31/31 [==============================] - 1s 6ms/step - loss: 1.3986 - mean_absolute_error: 0.3102 - val_loss: 1.1363 - val_mean_absolute_error: 0.2024
Epoch 2/7
31/31 [==============================] - 0s 2ms/step - loss: 0.9565 - mean_absolute_error: 0.1815 - val_loss: 0.7761 - val_mean_absolute_error: 0.1747
Epoch 3/7
31/31 [==============================] - 0s 2ms/step - loss: 0.6355 - mean_absolute_error: 0.1559 - val_loss: 0.4927 - val_mean_absolute_error: 0.1435
Epoch 4/7
31/31 [==============================] - 0s 2ms/step - loss: 0.3907 - mean_absolute_error: 0.1417 - val_loss: 0.2883 - val_mean_absolute_error: 0.1463
Epoch 5/7
31/31 [==============================] - 0s 2ms/step - loss: 0.2158 - mean_absolute_error: 0.1411 - val_loss: 0.1478 - val_mean_absolute_error: 0.1448
Epoch 6/7
31/31 [==============================] - 0s 2ms/step - loss: 0.1070 - mean_absolute_error: 0.1449 - val_loss: 0.0735 - val_mean_absolute_error: 0.1467
Epoch 7/7
31/31 [=================

- Regularizador Dropout

In [23]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

model_reg2 = Sequential()
model_reg2.add(Dense(150, input_shape=(5,), activation='relu'))
model_reg2.add(Dropout(0.5))
model_reg2.add(Dense(1, activation='relu'))

model_reg2.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
model_reg2.fit(X_train,y_train, epochs=7, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/7
31/31 [==============================] - 1s 5ms/step - loss: 0.2211 - mean_absolute_error: 0.4193 - val_loss: 0.0759 - val_mean_absolute_error: 0.2313
Epoch 2/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0785 - mean_absolute_error: 0.2234 - val_loss: 0.0568 - val_mean_absolute_error: 0.1979
Epoch 3/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0617 - mean_absolute_error: 0.2016 - val_loss: 0.0431 - val_mean_absolute_error: 0.1710
Epoch 4/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0514 - mean_absolute_error: 0.1833 - val_loss: 0.0363 - val_mean_absolute_error: 0.1609
Epoch 5/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0429 - mean_absolute_error: 0.1683 - val_loss: 0.0279 - val_mean_absolute_error: 0.1386
Epoch 6/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0364 - mean_absolute_error: 0.1567 - val_loss: 0.0221 - val_mean_absolute_error: 0.1221
Epoch 7/7
31/31 [=================

- Regularizador

In [24]:
from tensorflow.keras.layers import BatchNormalization

model_reg3 = Sequential()
model_reg3.add(Dense(150, input_shape=(5,), activation='relu'))
model_reg3.add(BatchNormalization())
model_reg3.add(Dense(1, activation='relu'))

model_reg3.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
model_reg3.fit(X_train,y_train, epochs=7, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/7
31/31 [==============================] - 2s 9ms/step - loss: 0.2394 - mean_absolute_error: 0.4201 - val_loss: 0.3698 - val_mean_absolute_error: 0.5817
Epoch 2/7
31/31 [==============================] - 0s 2ms/step - loss: 0.1356 - mean_absolute_error: 0.3045 - val_loss: 0.1721 - val_mean_absolute_error: 0.3794
Epoch 3/7
31/31 [==============================] - 0s 2ms/step - loss: 0.0764 - mean_absolute_error: 0.2171 - val_loss: 0.0483 - val_mean_absolute_error: 0.1863
Epoch 4/7
31/31 [==============================] - 0s 4ms/step - loss: 0.0352 - mean_absolute_error: 0.1434 - val_loss: 0.0350 - val_mean_absolute_error: 0.1569
Epoch 5/7
31/31 [==============================] - 0s 5ms/step - loss: 0.0171 - mean_absolute_error: 0.1024 - val_loss: 0.0324 - val_mean_absolute_error: 0.1494
Epoch 6/7
31/31 [==============================] - 0s 4ms/step - loss: 0.0154 - mean_absolute_error: 0.0978 - val_loss: 0.0321 - val_mean_absolute_error: 0.1489
Epoch 7/7
31/31 [=================